<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=948121060b3b74fe7350bcef54a64b39e8ad805b643e7923da7c7ca9df58738c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-18 12:45:07
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: 11.05 K (0.08%)
Current PnL: -27.65 L (-17.86%)
CY Booked + Current PnL: -13.36 L (-8.63%)
-------------------
Total profit:  1.42 L
Total loss:  -29.07 L
-------------------
Total Booked + Current PnL: 13.62 L (10.65%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.29 L (66.2%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,3.04,11.28,7.56,19.69,16131.0,21633.0,213375.0,0.44,63.0,X-MC,2.55,78.0,1.34,1.54,18.85,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.97,-19.95,25.03,0.09,20193.0,-20101.0,80676.0,87.19,28.0,M-SC,2.39,253.0,-1.00,0.58,5.99,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.57,-4.09,27.25,22.05,25608.0,-4005.0,93975.0,-17.34,49.0,H-SC,6.78,164.0,-0.16,0.68,36.60,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.42,-16.09,19.20,0.02,28122.0,-28090.0,146469.0,123.21,42.0,M-SC,13.67,234.0,-1.00,1.06,27.40,OX40N,NTT,PAINTS
43,ITC,452.00,0.30,-1.72,12.72,10.78,30043.0,-4129.0,236189.0,-41.56,45.0,X-LC,2.06,5.0,-0.14,1.70,3.43,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,11.09,-42.62,194.60,69.05,148589.0,-56710.0,76356.0,-53.70,63.0,H-SC,22.18,140.0,-0.38,0.55,17.76,SR,ATH,IT
6,ASIANTILES,137.00,6.09,-9.41,100.56,81.67,85454.0,-8832.0,84978.0,7644.44,64.0,L-SC,12.49,271.0,-0.10,0.61,64.56,XR,NTT,CERAMICS
82,VOLTAS,1530.00,3.04,11.28,7.56,19.69,16131.0,21633.0,213375.0,0.44,63.0,X-MC,2.55,78.0,1.34,1.54,18.85,XY25,NTT,AC
71,TANLA,1943.92,1.97,-36.45,245.68,119.69,422764.0,-98682.0,172079.0,-35.93,46.0,H-SC,7.82,156.0,-0.23,1.24,35.94,AR,ATH,IT
73,TCS,4389.96,1.65,-10.06,34.21,20.71,116377.0,-38053.0,340184.0,-22.65,68.0,X-LC,4.05,1.0,-0.33,2.45,13.67,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-3.04,-63.35,173.01,0.07,86991.0,-86896.0,50281.0,1743.43,51.0,L-SC,2.36,268.0,-1.00,0.36,25.05,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.0,-2.57,-46.08,222.99,74.15,196706.0,-75392.0,88213.0,-258.45,40.0,H-SC,22.86,149.0,-0.38,0.64,13.07,XR,NTT,CHEMICALS
16,CERA,9475.0,-2.44,-30.72,78.82,23.89,96055.0,-54037.0,121866.0,-33.17,40.0,H-SC,11.06,157.0,-0.56,0.88,5.25,OX40N,NTT,CERAMICS
14,CAMPUS,393.0,-2.29,-13.30,53.73,33.28,78158.0,-22310.0,145465.0,-30.19,32.0,M-SC,2.28,222.0,-0.29,1.05,12.75,XY24,NTT,FOOTWEAR
78,UNITDSPR,1644.0,-1.89,2.70,17.60,20.77,42320.0,6312.0,240456.0,-6.25,41.0,X-MC,2.37,62.0,0.15,1.73,9.80,X40N,NTT,BREWERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.21,-1.77,111.66,107.91,161938.0,-2616.0,145028.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.05,8.14,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.42,-16.09,19.20,0.02,28122.0,-28090.0,146469.0,123.21,42.0,M-SC,13.67,234.0,-1.00,1.06,27.40,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.97,-19.95,25.03,0.09,20193.0,-20101.0,80676.0,87.19,28.0,M-SC,2.39,253.0,-1.00,0.58,5.99,OX40N,NTT,DURABLES
66,SIS,528.00,0.05,-24.18,59.93,21.26,50454.0,-26844.0,84188.0,1986.01,50.0,H-SC,4.96,166.0,-0.53,0.61,13.75,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.09,-25.95,53.24,13.47,106311.0,-69984.0,199683.0,-69.48,32.0,H-SC,6.05,158.0,-0.66,1.44,1.53,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.88,8.06,59.42,72.27,108797.0,13650.0,183098.0,-7.44,69.0,M-LC,2.8,99.0,0.13,1.32,14.73,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.88,8.06,59.42,72.27,108797.0,13650.0,183098.0,-7.44,69.0,M-LC,2.80,99.0,0.13,1.32,14.73,XR,NTT,IT
25,FINCABLES,1641.55,1.21,-1.77,111.66,107.91,161938.0,-2616.0,145028.0,-20.15,57.0,M-SC,12.53,216.0,-0.02,1.05,8.14,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.38,-3.51,114.23,106.72,135947.0,-4324.0,119012.0,-52.15,38.0,H-SC,3.43,139.0,-0.03,0.86,19.05,AR,ATH,MISC
37,IEX,219.00,0.43,-4.42,55.51,48.63,106083.0,-8848.0,191106.0,-36.48,47.0,H-SC,13.54,136.0,-0.08,1.38,6.62,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.31,-3.49,40.53,35.63,81980.0,-7304.0,202270.0,-17.18,37.0,H-MC,2.77,119.0,-0.09,1.46,12.61,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.26,-18.76,91.37,55.47,180590.0,-45639.0,197647.0,-28.03,25.0,X-MC,13.93,64.0,-0.25,1.42,1.27,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.37,-10.83,43.67,28.11,78431.0,-21810.0,179600.0,-35.36,25.0,X-MC,13.46,55.0,-0.28,1.29,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.08,-24.41,103.06,53.50,235082.0,-73649.0,228102.0,-63.37,27.0,X-MC,3.79,58.0,-0.31,1.64,1.31,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.13,-39.49,123.44,35.20,96108.0,-50812.0,77858.0,2.09,29.0,X-SC,16.45,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
75,TMPV,600.00,-0.22,-29.17,73.61,22.98,117786.0,-65885.0,160013.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.15,0.64,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,1.43,-2.48,27.64,24.48,67889.0,-6237.0,245619.0,-81.32,42.0,X-SC,0.59,86.0,-0.09,1.77,20.09,X40N,NTT,MISC
12,BERGEPAINT,680.0,-1.10,-5.06,27.59,21.14,59552.0,-11494.0,215845.0,-11.79,36.0,X-MC,1.69,74.0,-0.19,1.56,21.66,XY24,NTT,PAINTS
20,DABUR,735.0,-0.37,-3.75,49.30,43.70,117712.0,-9302.0,238766.0,-11.62,34.0,X-MC,1.98,72.0,-0.08,1.72,10.74,XY24,BTT,FMCG
43,ITC,452.0,0.30,-1.72,12.72,10.78,30043.0,-4129.0,236189.0,-41.56,45.0,X-LC,2.06,5.0,-0.14,1.70,3.43,X40,NTT,FMCG
78,UNITDSPR,1644.0,-1.89,2.70,17.60,20.77,42320.0,6312.0,240456.0,-6.25,41.0,X-MC,2.37,62.0,0.15,1.73,9.80,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.14,-26.18,122.37,64.15,214943.0,-62301.0,175650.0,-56.75,31.0,X-SC,3.01,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.13,-39.49,123.44,35.20,96108.0,-50812.0,77858.0,2.09,29.0,X-SC,16.45,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,-0.37,-10.83,43.67,28.11,78431.0,-21810.0,179600.0,-35.36,25.0,X-MC,13.46,55.0,-0.28,1.29,0.00,X40,ATH,APPARELS
19,COLPAL,3726.84,0.08,-20.68,78.40,41.51,163778.0,-54465.0,208900.0,-5.42,35.0,X-MC,7.50,61.0,-0.33,1.51,0.08,XY25,ATH,FMCG
75,TMPV,600.00,-0.22,-29.17,73.61,22.98,117786.0,-65885.0,160013.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.15,0.64,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.20,-31.07,107.22,42.84,48045.0,-20196.0,44810.0,-54.82,38.0,X-SC,35.18,83.0,-0.42,0.32,1.38,XY24,NTT,MISC
59,RELAXO,1176.00,-0.05,-48.06,193.23,52.31,145690.0,-69763.0,75397.0,-44.73,40.0,X-SC,7.24,91.0,-0.48,0.54,0.92,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.13,-39.49,123.44,35.20,96108.0,-50812.0,77858.0,2.09,29.0,X-SC,16.45,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.99,-5.53,31.34,24.08,37585.0,-7022.0,119928.0,-13.19,31.0,X-SC,5.39,89.0,-0.19,0.86,13.00,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.71,-17.48,73.68,43.32,99026.0,-28472.0,134400.0,-29.05,33.0,X-SC,5.02,90.0,-0.29,0.97,3.19,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,1.65,-10.06,34.21,20.71,116377.0,-38053.0,340184.0,-22.65,68.0,X-LC,4.05,1.0,-0.33,2.45,13.67,X40,ATH,IT
41,INFY,1972.00,1.35,10.53,21.45,34.24,74184.0,32960.0,345848.0,-13.85,66.0,X-LC,4.57,2.0,0.44,2.49,19.69,X40,NTT,IT
75,TMPV,600.00,-0.22,-29.17,73.61,22.98,117786.0,-65885.0,160013.0,-26.74,30.0,X-LC,6.69,3.0,-0.56,1.15,0.64,XY24,NTT,AUTO
81,VBL,671.64,0.31,-4.60,42.21,35.67,127190.0,-14515.0,301327.0,-15.92,51.0,X-LC,4.04,4.0,-0.11,2.17,8.62,X40N,ATH,FMCG
43,ITC,452.00,0.30,-1.72,12.72,10.78,30043.0,-4129.0,236189.0,-41.56,45.0,X-LC,2.06,5.0,-0.14,1.70,3.43,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.62,-38.06,114.54,32.89,54776.0,-29384.0,47823.0,-690.94,51.0,L-MC,6.26,259.0,-0.54,0.34,31.70,XR,NTT,BANKS
6,ASIANTILES,137.00,6.09,-9.41,100.56,81.67,85454.0,-8832.0,84978.0,7644.44,64.0,L-SC,12.49,271.0,-0.10,0.61,64.56,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.57,-4.09,27.25,22.05,25608.0,-4005.0,93975.0,-17.34,49.0,H-SC,6.78,164.0,-0.16,0.68,36.60,XR,ATH,FINANCE
13,BSOFT,831.70,0.31,-21.46,91.99,50.79,101618.0,-30187.0,110466.0,0.89,65.0,H-SC,6.60,151.0,-0.30,0.80,28.62,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.42,-16.09,19.20,0.02,28122.0,-28090.0,146469.0,123.21,42.0,M-SC,13.67,234.0,-1.00,1.06,27.40,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.31,-21.46,91.99,50.79,101618.0,-30187.0,110466.0,0.89,65.0,H-SC,6.60,151.0,-0.30,0.80,28.62,XR,ATH,IT
84,WIPRO,420.00,0.88,8.06,59.42,72.27,108797.0,13650.0,183098.0,-7.44,69.0,M-LC,2.80,99.0,0.13,1.32,14.73,XR,NTT,IT
41,INFY,1972.00,1.35,10.53,21.45,34.24,74184.0,32960.0,345848.0,-13.85,66.0,X-LC,4.57,2.0,0.44,2.49,19.69,X40,NTT,IT
61,ROUTE,2227.21,11.09,-42.62,194.60,69.05,148589.0,-56710.0,76356.0,-53.70,63.0,H-SC,22.18,140.0,-0.38,0.55,17.76,SR,ATH,IT
73,TCS,4389.96,1.65,-10.06,34.21,20.71,116377.0,-38053.0,340184.0,-22.65,68.0,X-LC,4.05,1.0,-0.33,2.45,13.67,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.39
1,25,44.88
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.78
MC    30.95
LC    24.25
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.56
X40      23.15
X40N     12.99
XR        9.80
XY25      9.15
AR        9.00
OX40N     7.52
SR        1.04
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.46
H-SC    23.76
X-LC    20.79
M-SC    11.57
X-SC     8.01
H-MC     4.81
L-SC     1.44
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.32,-8.54,43.45
IT,13.22,-14.60,72.22
FINANCE,9.43,-21.67,74.58
MISC,7.29,-28.44,80.98
ELECTRICAL,5.99,-12.89,54.25
PAINTS,5.82,-14.62,31.64
INSURANCE,4.80,-1.76,36.59
PHARMA,4.15,-2.29,34.49
AUTO,2.80,-34.59,81.25


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3234332.0,21
AR,1323664.0,10
XR,1302751.0,13
X40,1010092.0,14
X40N,882778.0,9
OX40N,748091.0,10
XY25,372695.0,6
SR,276580.0,2
MH,77708.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3683955.0,25
M-SC,1450949.0,15
X-MC,1432853.0,16
X-LC,841522.0,11
X-SC,806301.0,8
H-MC,412469.0,3
L-SC,261303.0,3
M-LC,108797.0,1
H-LC,79472.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1295207.0      6
           AR         900712.0      5
M-SC       XY24       831300.0      6
H-SC       XR         788389.0      7
X-MC       X40        494160.0      7
           XY24       412346.0      3
           X40N       346438.0      4
H-SC       OX40N      345359.0      4
X-LC       X40        322809.0      5
M-SC       OX40N      315741.0      5
X-SC       X40N       312605.0      3
           XY24       300573.0      3
H-SC       SR         276580.0      2
X-LC       X40N       223735.0      2
H-MC       AR         216049.0      2
X-LC       XY24       198486.0      2
H-MC       XY24       196420.0      1
X-SC       X40        193123.0      2
X-MC       XY25       179909.0      2
M-SC       XR         176477.0      2
L-SC       XR         174312.0      2
M-SC       AR         127431.0      2
M-LC       XR         108797.0      1
X-LC       XY25        96492.0      2
L-SC       OX40N       86991.0      1
H-LC       AR          79472.0      1
H-SC       MH          77708.0      1
M-MC       XY25        59537.0      1
L-MC       XR          54776.0      1
L-LC       XY25        36757.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
